# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Final Project Milestone 3 - LexRank Model

**Harvard University**<br/>
**Spring 2020**<br/>
**Group 32** 

<hr style="height:2pt">

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
# same as EDA
# libraries
import json
import lzma
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from IPython.core.display import display, HTML
import re
from nltk.tokenize import RegexpTokenizer
import datetime as dt
import math
from collections import Counter
from rouge import Rouge 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## <a id='0'>Content</a>

- <a href='#1'>1. Introduction</a> 
- <a href='#2'>2. Citation Filter</a>
- <a href='#3'>3. Importing the legal documents</a>
- <a href='#4'>4. LexRank Implementation</a>
- <a href='#5'>5. Examples</a>
- <a href='#6'>6. Challenges to thematic segmentation</a>
- <a href='#7'>7. Conclusion</a>

## <a id='1'>1. Introduction</a>

In order to summarize the legal texts, we also test the LexRank model, which is one of the ways for extractive summarization.

There are two ways to summarize texts using NLP; extractive and abstractive. Since the extractive approach chooses sentences from the original text, it is less likely to create an incomprehensible summary and less likely to output a gramatically incorrect summary. However, in the extractive approach, you cannot use words that are not in the original text, so it is not possible to paraphrase or to use conjunctions to make summary more easy to read. In contrast, the abstractive allows you to summarize more flexibly because you are free to use words that are not in the original sentences. In addition, you can choose the length of summary. However, the disadvantage in this approach is that it is difficult to produce "natural" sentences. For this project, we decided to take the extractive way to ensure that important information is included in the summary of the legal documents.

The LexRank model works almost the same as Google Search -- it uses sentences as a node and similarity as edge/weight. For details, please refer to [Erkan and Radev (2004)](https://www.aaai.org/Papers/JAIR/Vol22/JAIR-2214.pdf).

## <a id='2'>2. Citation Filter</a>

According to [Farzindar and Lapalme (2004)](https://www.aclweb.org/anthology/W04-1006.pdf), the citations account for a large part of the text of the judgment, but they are not considered relevant for the summary. Therefore, I removed sentences that include specific abbreviation or words.

In [3]:
import re

# Reference: https://library.csustan.edu/apalegal

def citation_filter(text_list):
    new_list = []
    
    for i in range(len(text_list)):
        sentence = text_list[i]
        
        if re.search('\(\d\d\d\d\)', sentence) != None:
            pass
        elif re.search('v\.', sentence) != None:
            pass
        elif re.search('vs\.', sentence) != None:
            pass
        elif re.search('§', sentence) != None:
            pass
        elif re.search('R\.', sentence) != None:
            pass
        elif re.search('Rule', sentence) != None:
            pass
        # Ark. = Arkansas
        elif re.search('Ark\.', sentence) != None:
            pass
        else:
            new_list.append(sentence)
            
    return new_list

## <a id='3'>3. Importing the legal documents</a>

In [4]:
# same as EDA
# defining a fucnction to remove \n and HTML tags
def text_cleaner(text):
    text_divided = text.splitlines()
    text_divided_clean = " ".join(text_divided)
    return text_divided_clean

In [5]:
# same as EDA
# The file size for some states are too large to open into memory
# This function loads individual cases into memory, parses headnotes and 
# opinions, cleans the text, tokenizes the text, and returns counts of tokens
# for each case.

tokenizer = RegexpTokenizer('\s+', gaps=True)

def get_counts(state):
    cases = []
    with lzma.open("../" + state + '-text/data/data.jsonl.xz', 'r') as jsonl_file:
        for case in jsonl_file:
            c = json.loads(str(case, 'utf-8'))

            date = c['decision_date']
            
            headnotes = text_cleaner(c['casebody']['data']['head_matter'])
            headnotes_tokenized = tokenizer.tokenize(headnotes)
            num_headnotes = len(headnotes_tokenized)

            opinions = c['casebody']['data']['opinions']
            if opinions == []:
                num_opinions = 0
            else:
                opinions = text_cleaner(opinions[0]['text'])
                opinions_tokenized = tokenizer.tokenize(opinions)
                num_opinions = len(opinions_tokenized)
            cases.append({'date':date, 'num_headnotes':num_headnotes, 'headnotes': headnotes, 'num_opinions':num_opinions, 'opinions':opinions})
        return pd.DataFrame(cases)

In [6]:
%%time

# use North Carolina data as an example
states = ['North Carolina']
counts_nc = get_counts(states[0])

CPU times: user 1min 15s, sys: 1.05 s, total: 1min 16s
Wall time: 1min 16s


## <a id='4'>4. LaxRank Implementation</a>

In [7]:
# compute term frequency
# Reference: http://www.tfidf.com/
# sentence: dictionary
def termfreq(sentence):

    dict_counts = Counter(sentence)
    max_tf = max(dict_counts.values())

    tf_dict = {}
    
    # create a dictionary with word + term frequncy
    for word, tf in dict_counts.items():
        tf_dict[word] = tf / max_tf
    
    # Output Example: {'word': tf, 'word': tf, ...}
    return tf_dict

# Create a dictionary with words and their IDF: Inverse Document Frequncy
# Reference: http://www.tfidf.com/
# list_sentences: list
def compute_idf(list_sentences):
    
    idf_dict = {}
    sentences_len = len(list_sentences)

    for sentence in list_sentences:
        for word in sentence:
            if word not in idf_dict:
                
                # if not in idf_dict, calculate idf and append it to the dictionary
                number_appearance = 0
                for sen in list_sentences:
                    if word in sen:
                        number_appearance += 1
                idf_dict[word] = math.log(sentences_len / number_appearance)
                
    return idf_dict

In [8]:
# Reference: http://www.cs.cmu.edu/afs/cs/project/jair/pub/volume22/erkan04a-html/erkan04a.html
# Chapter 3 : Centrality-based Sentence Salience
def idf_modified_cosine(list_sentences, list_sentence_x, list_sentence_y):
    
    dict_x = termfreq(list_sentence_x)
    dict_y = termfreq(list_sentence_y)
    idf_dict = compute_idf(list_sentences)

    set_unique_words1 = set(list_sentence_x)
    set_unique_words2 = set(list_sentence_y)

    words_xy = set_unique_words1 & set_unique_words2
    
    numerator = 0
    for word in words_xy:
        numerator = numerator + dict_x[word] * dict_y[word] * ((idf_dict[word])**2)
        
    denominator_left_quad = 0
    for word in set_unique_words1:
        denominator_left_quad = denominator_left_quad + ((dict_x[word] * idf_dict[word]) ** 2)
        
    denominator_right_quad = 0
    for word in set_unique_words2:
        denominator_right_quad = denominator_right_quad + ((dict_y[word] * idf_dict[word]) ** 2)
    
    denominator_left = math.sqrt(denominator_left_quad)
    denominator_right = math.sqrt(denominator_right_quad)
    
    if denominator_left == 0 or denominator_right == 0:
        print("Error! 0 in denominator!")
    else:
        return numerator / (math.sqrt(denominator_left) * math.sqrt(denominator_right))

In [9]:
# Reference: http://www.cs.cmu.edu/afs/cs/project/jair/pub/volume22/erkan04a-html/erkan04a.html
# 3.2 Eigenvector Centrality and LexRank
# computing the stationary distribution of a Marcov chain
def power_method(M, N, eps):

    # initialization
    p = np.full((N,), 1/N)

    delta = 999

    while delta >= eps:
        p_t = np.dot(np.transpose(M), p)
        delta = np.linalg.norm(p_t - p)
        p = p_t

    return p

In [10]:
# Reference: http://www.cs.cmu.edu/afs/cs/project/jair/pub/volume22/erkan04a-html/erkan04a.html
# 3.2 Eigenvector Centrality and LexRank, Algorithm #3

# Lexrank: summarizing sentences
# computing Lexrank Scores
def lex_rank(list_sentences, n, t):

    cosinematrix = np.zeros((n, n))
    degree = np.zeros((n,))

    for i in range(n):
        for j in range(n):
            cosinematrix[i][j] = idf_modified_cosine(list_sentences, list_sentences[i], list_sentences[j])
            if cosinematrix[i][j] > t:
                cosinematrix[i][j] = 1
                degree[i] += 1
            else:
                cosinematrix[i][j] = 0

    for i in range(n):
        for j in range(n):
            cosinematrix[i][j] = cosinematrix[i][j] / degree[i]

    L = power_method(cosinematrix, n, t)

    return zip(list_sentences, L)

## <a id='5'>5. Examples</a>

### Example 1

In [11]:
test_opinions = counts_nc.iloc[97597,4]

In [12]:
test_opinions

'MORRIS, Chief Judge. The sole issue on appeal is whether claimants, were discharged because of misconduct associated with their work and are thus disqualified from receiving unemployment benefits. Findings of fact of the Commission are conclusive if supported by the evidence, and judicial review is limited to determining whether errors of law have been committed. G.S. 96-15(i). The findings of fact to which appellant excepts are as follows: 5. The employer alleged they were discharged as a result of having found gambling for money while on company property and during a work day. 6. Each of the parties (claimants) involved denies having done any gambling for money. Each party (claimant) does agree that they were playing cards for a game or two, but not with any betting for money involved. Someone had found an incomplete deck of cards and while waiting for job assignments the four claimants fooled around playing a hand or two of cards. 7. The claimants were not playing cards for money a

In [13]:
# Reference: https://nlpforhackers.io/splitting-text-into-sentences/

from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
 
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(test_opinions)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [14]:
test_opinions_list = tokenizer.tokenize(test_opinions)

In [15]:
def text_summarizer(text, n, t=1):
    """
    n: number of sentences
    t: error tolerance
    """
    words_list = []
    for i in range(len(text)):
        words = text[i].split()
        words_list.append(words)
    zipped = lex_rank(words_list, len(text), t)
    unzipped = list(zip(*zipped))
    scores = np.array(unzipped[1])
    highest_index = scores.argsort()[-n:][::-1]
    summarized = []
    high_scores = []
    for i in range(len(highest_index)):
        sentence = text[i]
        score = scores[i]
        high_scores.append(score)
        summarized.append(sentence)     
    print("\nOriginally", len(text), "sentences\n")
    print("Summarized in", n, "sentences\n")
    print("Summarized:  ", summarized,"\n")
    return summarized

In [16]:
%%time

# 616 words
result1 = text_summarizer(citation_filter(test_opinions_list), 7, 0.1)


Originally 47 sentences

Summarized in 7 sentences

Summarized:   ['MORRIS, Chief Judge.', 'The sole issue on appeal is whether claimants, were discharged because of misconduct associated with their work and are thus disqualified from receiving unemployment benefits.', 'Findings of fact of the Commission are conclusive if supported by the evidence, and judicial review is limited to determining whether errors of law have been committed.', 'G.S. 96-15(i).', 'The findings of fact to which appellant excepts are as follows: 5.', 'The employer alleged they were discharged as a result of having found gambling for money while on company property and during a work day.', '6.'] 

CPU times: user 7.19 s, sys: 3.96 ms, total: 7.2 s
Wall time: 7.2 s


#### Rouge Scores

In [17]:
string1 = ''.join(result1)

rouge = Rouge()
scores = rouge.get_scores(string1, counts_nc.iloc[97597,2])
print("Original Headnote: ", counts_nc.iloc[97597,2], "\n\n")
print("Rouge F1 Score:", scores[0]['rouge-1']['f'], "Rouge F2 Score:", scores[0]['rouge-2']['f'])

Original Headnote:  IN THE MATTER OF: WALTER KIDDE & COMPANY, INC., Post Office Box 509, Mebane, North Carolina 27302 v. JACK D. BRADSHAW, Route 1, Box 365, Mebane, North Carolina 27302, SS. No. [ XXX-XX-XXXX ], Docket No. 4235 G; ED FISHER, Route 8, Box 123, Burlington, North Carolina 27215, SS. No. [ XXX-XX-XXXX ], Docket No. 4242 G; GRADY L. HUNDLEY, Route 2, Box 576, Graham, North Carolina 27253, SS. No. [ XXX-XX-XXXX ], Docket No. 4247 G; DAVID A. TUTTLE, Route 8, Box 165, Burlington, North Carolina 27215, SS. No. [ XXX-XX-XXXX ], DOCKET No. 4277 G; AND EMPLOYMENT SECURITY COMMISSION OF NORTH CAROLINA, Post Office Box 25903, Raleigh, North Carolina 27611 No. 8115SC524 (Filed 6 April 1982) Haynsworth, Baldwin, Miles, Johnson, Greaves and Edwards, by Charles P. Roberts, for plaintiff appellant. Gail C. Arneke and C. Coleman Billingsley, Jr., for Employment Security Commission of North Carolina, appellee. 


Rouge F1 Score: 0.06530611770095829 Rouge F2 Score: 0.0


#### Example 2

In [18]:
test2_opinions = counts_nc.iloc[97577,4]
test2_opinions_list = tokenizer.tokenize(test2_opinions)

In [19]:
%%time

result2 = text_summarizer(citation_filter(test2_opinions_list), 15, 0.1)


Originally 26 sentences

Summarized in 15 sentences

Summarized:   ['Walker, J. Plaintiff alleged ownership, under a grant from the State t.o himself, of a tract of land containing 74 acres, more or less, on the north side of Pamlico Eiver and the west side of Bath Creek and designated on the court map by the figures 1, 2, 3, 4, 5, and back to 1, and on which the trespass was alleged to have been committed by cutting timber.', 'Defendant denied plaintiff’s title upon two grounds: 1.', "That it had acquired title to the land under a deed of Jesse C. Bryan to Thomas D. Beasley, dated 19 March, 1846, and a deed from James E. Shepherd, commissioner to sell the lands of the said Thomas D. Beasley, who had died, dated '2 June, 1882, and adverse possession under these deeds.", 'Plaintiff claimed that the line 2 to 5, as shown on the map, is the western boundary of the deed of Bryan to Beasley, while the defendant contended that its western boundary is the line B, 0, 3.', '2.', "There was a d

#### Rouge Scores

In [20]:
string2 = ''.join(result2)

rouge = Rouge()
scores = rouge.get_scores(string2, counts_nc.iloc[97577,2])
print("Original Headnote: ", counts_nc.iloc[97577,2], "\n\n")
print("Rouge F1 Score:", scores[0]['rouge-1']['f'], "Rouge F2 Score:", scores[0]['rouge-2']['f'])

Original Headnote:  JOHN D. ELLIOTT v. ROANOKE RAILROAD AND LUMBER COMPANY. (Filed 22 September, 1915.) 1. Trespass — Title—Burden of Proof. The weakness of the defendant’s title to land will not avail the plaintiff in an action of trespass involving title, for he must recover, if at all, upon the strength of his own title. 2. Same — State Grants — Deeds and Conveyances — Color—Plaintiff’s Evidence. Where the plaintiff’s own evidence, in an action of trespass on lands involving title, tends to show sufficient adverse possession of the defendant under color to take the title out of the State and ripen it in defendant, or in one under whom he claims, and the plaintiff is claiming the locus in quo by grant from the State, issued after the title had ripened, he cannot recover. Appeal by plaintiff from Justice, J., at February Term, 1915, of BEAUFORT. Civil action for trespass on land. Daniel & Warren and Bryan & Stewart for plaintiff. Small, McLean, Bragaw & Rodman for defendants. 


Rouge

#### Example 3

In [21]:
test3_opinions = counts_nc.iloc[97576,4]
test3_opinions_list = tokenizer.tokenize(test3_opinions)

In [22]:
%%time

result3 = text_summarizer(citation_filter(test3_opinions_list), 20, 0.1)


Originally 80 sentences

Summarized in 20 sentences

Summarized:   ['Claek, C. J. Tbe General Assembly of 1913, chapters 248 and 276 placed tbe county of Pender within tbe public policy now prevailing over nine-tentbs of tbe territory of tbis State, under wbat is known as tbe “no-fence” law, by wbicb stock are not allowed to run at large on tbe lands of others than tbeir owners, and requires that such owners shall fence up tbeir stock instead of other people fencing them out in order to protect tbeir crops.', "Tbe Legislature, by Public-Local Laws 1915, chapters 116 and 505, permitted tbe people of a certain part of Pender County to decide by vote whether they should return to tbe former system of letting' stock run at large, but made such provision, if adopted by such vote, dependent upon tbe condition precedent, that tbe change should not take effect until a fence should be constructed by such territory to prevent the stock therein trespassing upon tbe people of tbe adjoining counti

#### Rouge Scores

In [23]:
string3 = ''.join(result3)

rouge = Rouge()
scores = rouge.get_scores(string3, counts_nc.iloc[97576,2])
print("Original Headnote: ", counts_nc.iloc[97576,2], "\n\n")
print("Rouge F1 Score:", scores[0]['rouge-1']['f'], "Rouge F2 Score:", scores[0]['rouge-2']['f'])

Original Headnote:  D. H. MARSHBURN et als. v. ISAAC JONES et als. (Filed 27 November, 1918.) 1. Statutes — Stock—No-Fence Law — “Change of Fence.” The requirement of Revisal, sec. 1675, that the counties therein named may withdraw from the operation of the no-fence law, upon the conditions specified therein, if funds are provided by a tax levy, etc., for “changing the fence,” is to provide against trespass by the running at large of stock into no-fence territory, and contemplates the change from the one system to the other; and the position is untenable that the statute is inapplicable when the fence has long since been lawfully removed or destroyed. 2. Statutes — Repealing Statutes — Conflict—Stock—No-Fence Law — Fences. Where a statute amends Revisal, sec. 1675, by adding a part of another county to those therein named as having the right to withdraw from the stock law under certain conditions, and makes the .building of the fence around the outer boundaries of the proposed district

#### Rouge F1 & F2 With the first 100 observations

In [24]:
def text_summarizer_simple(text, n, t=1):
    """
    n: number of sentences
    t: error tolerance
    """
    words_list = []
    for i in range(len(text)):
        words = text[i].split()
        words_list.append(words)
    zipped = lex_rank(words_list, len(text), t)
    unzipped = list(zip(*zipped))
    scores = np.array(unzipped[1])
    highest_index = scores.argsort()[-n:][::-1]
    summarized = []
    high_scores = []
    for i in range(len(highest_index)):
        sentence = text[i]
        score = scores[i]
        high_scores.append(score)
        summarized.append(sentence)     
    return summarized

In [25]:
%%time

# use texts shorter than 2000 to save computation

score_100_f1 = 0
score_100_f2 = 0

counter = 0
counter_100 = 0

while counter_100 < 100:
    
    row_num = 57000 + counter
    
    if counts_nc.iloc[row_num,3] > 2000:
        counter += 1
    else:
        test100_opinions = counts_nc.iloc[row_num,4]
        test100_opinions_list = tokenizer.tokenize(test100_opinions)

        result100 = text_summarizer_simple(citation_filter(test100_opinions_list), 20, 0.1)

        string100 = ''.join(result100)

        rouge = Rouge()
        scores = rouge.get_scores(string100, counts_nc.iloc[row_num,2])

        score_100_f1 += scores[0]['rouge-1']['f']
        score_100_f2 += scores[0]['rouge-2']['f']
        
        counter += 1
        counter_100 += 1
        
print("Average Rouge F1 Score:", score_100_f1 / 100, "\n")
print("Average Rouge F2 Score:", score_100_f2 / 100)

Average Rouge F1 Score: 0.3725000474930513 

Average Rouge F2 Score: 0.13328381119168312
CPU times: user 13min 11s, sys: 504 ms, total: 13min 12s
Wall time: 13min 12s


## <a id='6'>6. Challenges to thematic segmentation</a>

In [Farzindar and Lapalme (2004)](https://www.aclweb.org/anthology/W04-1006.pdf), thematic segmentation by linguistic markars is introduced as a way to more easily summarize legal documents. This is because we can follow the structure of legal documets, and we are less likely to miss important information. We tried to implement the thematic segmentation, but we observed, for example, linguistic markars for conclusion in the beginning of a legal document. Therefore, we decided not to use thematic segmentation for summarization.

In [26]:
# Reference: https://www.aclweb.org/anthology/W04-1006.pdf

markars_introduction = ['application for judicial review', 'application to review a decision', 'motion filed by', 'Statement of Claim']

markars_context = ["advise","indicate","concern","request"]

markars_juridical_analysis = ['this court','In reviewing',
                            'Pursuant to section','As I have stated','In the present case']

markars_conclusion = ['note','accept','summarise','scrutinize','think','say','satisfy','discuss','conclude','find','believe','reach','persuade',
                      'agree','indicate','review']

In [27]:
markars = [markars_introduction, markars_context, markars_juridical_analysis, markars_conclusion]

def markar_detector(text_list):
    for i in range(len(text_list)):
        sentence = text_list[i]
        
        for markar in markars:
            for j in range(len(markar)):
                markar_word = markar[j]
                TF = markar_word in sentence
                if TF == True:
                    if markar == markars_context:
                        type_markar = "context"
                        print("Linguistic markar '"+markar_word+"' detected! Sentence #", i, "of "+str(len(text_list))+". This is",type_markar,"markar.")
                    elif markar == markars_juridical_analysis:
                        type_markar = "juridical analysis"
                        print("Linguistic markar '"+markar_word+"' detected! Sentence #", i, "of "+str(len(text_list))+". This is",type_markar,"markar.") 
                    else:
                        type_markar = "conclusion"
                        print("Linguistic markar '"+markar_word+"' detected! Sentence #", i, "of "+str(len(text_list))+". This is",type_markar,"markar.")   

In [29]:
test_text4 = counts_nc.iloc[10,4]
test_text_list4 = tokenizer.tokenize(test_text4)

markar_detector(test_text_list4)

Linguistic markar 'find' detected! Sentence # 6 of 615. This is conclusion markar.
Linguistic markar 'conclude' detected! Sentence # 7 of 615. This is conclusion markar.
Linguistic markar 'find' detected! Sentence # 9 of 615. This is conclusion markar.
Linguistic markar 'indicate' detected! Sentence # 29 of 615. This is context markar.
Linguistic markar 'indicate' detected! Sentence # 29 of 615. This is conclusion markar.
Linguistic markar 'agree' detected! Sentence # 43 of 615. This is conclusion markar.
Linguistic markar 'concern' detected! Sentence # 44 of 615. This is context markar.
Linguistic markar 'say' detected! Sentence # 61 of 615. This is conclusion markar.
Linguistic markar 'indicate' detected! Sentence # 75 of 615. This is context markar.
Linguistic markar 'indicate' detected! Sentence # 75 of 615. This is conclusion markar.
Linguistic markar 'agree' detected! Sentence # 106 of 615. This is conclusion markar.
Linguistic markar 'note' detected! Sentence # 111 of 615. This 

## <a id='7'>7. Conclusion</a>

Although the LexRank Model summarizes the legal documents with a reasonably long headnote well (Example 2 and 3), the average Rouge F1 and F2 are not so good because we fixed the length of summary to be 20 and it is not flexible. There is a couple of ways to improve the LexRank model.

- We might miss conclusion (juridical decision) because we do not implement thematic segmentation and the model cannot recognize the conclusion sentences as important.
- Regarding thematic segmentation, we should have better lists of linguistic markars. In addition, we could set a stop condition for thematic segmentation (e.g. if we observe conclusion markars N times in a row, we recognize all the texts after that as conclusion.)
- We should also think about the citation filter as headnotes include citations especially in conclusion. We should not the citation filter for summarizing conclusion. (But in order to achieve it, we need to implement thematic segmentation.)